In [39]:
#Dinesh Karamchandani A20407484

In [40]:
import pandas as pd                                    # panda's nickname is pd
import json                                  # numpy as np
df1 = [json.loads(line) for line in open('SPM587SP18issues.json')]

In [41]:
from elasticsearch import Elasticsearch, helpers
es = Elasticsearch()

In [42]:
#print 1st record of dataframe of json
df1[:1]

[{'Author': 'HSP18SCM50W',
  'State': 'closed',
  'closed_at': '2018-04-22',
  'created_at': '2018-04-20',
  'issue_number': 475,
  'labels': ['Category:Inquiry',
   'DetectionPhase:Field',
   'OriginationPhase:Coding',
   'Priority:High',
   'Status:inProgress']}]

In [43]:
#build the index
actions = []
for data in df1:
    action = {
        "_index": "issues_1",
        "_type": "issues",
        "_id": data["issue_number"],
        "_source":data
        
        
        # iterate on keys and if keys is list then split 
        
    }
    actions.append(action)
#actions
helpers.bulk(es,actions)

(255, [])

In [44]:
#Part 1) query to get all the records
doc = {
        'size': 10000,
        'query':{
            'match_all': {}
        }
}

all_results = es.search(index ='issues_1', body =doc,scroll = '1h')

In [45]:
#getting scroll id and scroll size
sid = all_results['_scroll_id']
scroll_size = all_results['hits']['total']

In [46]:
sid

'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAB-FjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAfxYzeFpmaERmcFJhNkdCVDFkMVRNZjZnAAAAAAAAAIAWM3haZmhEZnBSYTZHQlQxZDFUTWY2ZwAAAAAAAACBFjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAghYzeFpmaERmcFJhNkdCVDFkMVRNZjZn'

In [47]:
scroll_size

255

In [48]:
all_results

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAAB-FjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAfxYzeFpmaERmcFJhNkdCVDFkMVRNZjZnAAAAAAAAAIAWM3haZmhEZnBSYTZHQlQxZDFUTWY2ZwAAAAAAAACBFjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAghYzeFpmaERmcFJhNkdCVDFkMVRNZjZn',
 '_shards': {'failed': 0, 'skipped': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': '470',
    '_index': 'issues_1',
    '_score': 1.0,
    '_source': {'Author': 'RSP18SCM19N',
     'State': 'closed',
     'closed_at': '2018-04-14',
     'created_at': '2018-04-14',
     'issue_number': 470,
     'labels': ['Category:Enhancement',
      'DetectionPhase:Testing',
      'OriginationPhase:Design',
      'Priority:Major',
      'Status:Completed']},
    '_type': 'issues'},
   {'_id': '452',
    '_index': 'issues_1',
    '_score': 1.0,
    '_source': {'Author': 'YSP18SCM40K',
     'State': 'closed',
     'closed_at': '2018-04-13',
     'created_at': '2018-04-13',
     'issue_number': 452,
     'labels': ['Address:225 S Canal St Chicago 

In [49]:
#writing function to get list of locations for folium heatmap from the results
def get_final_location(all_results, scroll_size,sid):
    count = 0 
    count2,count3 =0,0
    final_location = []
    final_issue_no = []
    final_issue_no2 = []
    while(scroll_size>0):    
        for doc in all_results['hits']['hits']: #[1]['_source']['labels']:
            #print((i.split(':'))[0])
            location_ll = []
            location_dict = {}
            results = doc['_source']['labels']
            count = count+1
            for i in results:
                #print(i)
                #print("In here")
                label_name = (i.split(':'))[0].strip().title()
                #label_value = (i.split(':'))[1]
                if (label_name == 'Latitude' or label_name == 'Longitude' and (i.split(':'))[1] != None): 
                    label_value = (i.split(':'))[1].strip()
                    #print(label_name,label_value)
                    location_dict[label_name] = float(label_value)
                    final_issue_no.append(doc['_source']['issue_number'])
                    #count2=count2+1
                    #location.append(float(label_value))                
                    #print("Inside for location_dict", location_dict)
            #print("Outside for")
            if('Longitude' in location_dict and 'Latitude' in location_dict):
                count3=count3+1
                final_issue_no2.append(doc['_source']['issue_number'])
                final_location.append([location_dict['Latitude'],location_dict['Longitude']])
                #print("Final location", final_location)
            #print("Outside for location-dict", location_dict)
        all_results = es.scroll(scroll_id = sid, scroll = '2m')
        sid = all_results['_scroll_id']
        scroll_size = len(all_results['hits']['hits'])  
    #print("records fetched", count)
    #print("records fetched", count2)
    print("records fetched", count3)
    return final_location

final_location = get_final_location(all_results, scroll_size,sid)
#although issues are 253, only 142 have location against them hence we will only 142 records in the heatmap

records fetched 143


In [50]:
#printing final_locations first 10 records
final_location[:10]

[[41.878693, -87.638924],
 [41.838897, -87.646804],
 [41.858415, -87.660926],
 [41.89302, -87.631556],
 [41.878693, -87.638924],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.847095, -87.616767],
 [41.877817, -87.631247],
 [41.89323, -87.617419]]

In [51]:
import folium 
from folium import plugins
print(folium.__version__)

0.5.0


In [52]:
#define function for getting folium heatmap

def get_folium_heatmap(final_location):
    without_condition_heatmap = folium.Map([41.878693, -87.638924],zoom_start = 11)
    return without_condition_heatmap.add_child(plugins.HeatMap(final_location,radius=15))
    #return without_condition_heatmap

heatmap = get_folium_heatmap(final_location)


# Part1 Folium heatmap for all the issues

In [53]:
heatmap

# Part2. 1) DetectionPhase is Field AND Priority is Critical

In [54]:
doc = {
        'size': 10000,
        'query':{
            'bool': {
                    'must' : [
                        
                        {'match' : {'labels':'DetectionPhase:Field'}},
                        {'match' : {'labels':'Priority:Critical'}}
                    ]
            }
            
        }
}

all_results = es.search(index ='issues_1', body =doc,scroll = '1h')

In [55]:
sid = all_results['_scroll_id']
scroll_size = all_results['hits']['total']
print("Sid:", sid)
print("Scroll size:", scroll_size)

Sid: DnF1ZXJ5VGhlbkZldGNoBQAAAAAAAACDFjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAhBYzeFpmaERmcFJhNkdCVDFkMVRNZjZnAAAAAAAAAIUWM3haZmhEZnBSYTZHQlQxZDFUTWY2ZwAAAAAAAACGFjN4WmZoRGZwUmE2R0JUMWQxVE1mNmcAAAAAAAAAhxYzeFpmaERmcFJhNkdCVDFkMVRNZjZn
Scroll size: 4


In [56]:
final_location = get_final_location(all_results, scroll_size,sid)

#only 3 records were collected as one of the records only has only latitude and no longitude - issue#12
final_location

records fetched 3


[[41.89323, -87.617419], [41.917164, -87.686965], [41.891551, -87.607375]]

# Part 2. 1) Heatmap for DetectionPhase is Field AND Priority is Critical

In [57]:
heatmap = get_folium_heatmap(final_location)
heatmap

# Part 2. 2) DetectionPhase is Field AND Status is Completed

In [58]:
doc = {
        'size': 10000,
        'query':{
            'bool': {
                    'must' : [
                        
                        {'match' : {'labels':'DetectionPhase:Field'}},
                        {'match' : {'labels':'Status:Completed'}}
                    ]
            }
            
        }
}

all_results = es.search(index ='issues_1', body =doc,scroll = '1h')

In [59]:
sid = all_results['_scroll_id']
scroll_size = all_results['hits']['total']
scroll_size

4

In [60]:
final_location = get_final_location(all_results, scroll_size,sid)

final_location
#40.170101, -92.177847 located far from chicago hence not in map

records fetched 4


[[40.170101, -92.177847],
 [41.891551, -87.607375],
 [41.89323, -87.617419],
 [41.853136, -87.63316]]

# Part 2. 2) Heatmap for DetectionPhase is Field AND Status is Completed

In [61]:
heatmap = get_folium_heatmap(final_location)
heatmap

# Part 2. 3)DetectionPhase is Field AND Priority is Critical AND Status is Approved

In [62]:
doc = {
        'size': 10000,
        'query':{
            'bool': {
                    'must':[
                        {'match' : {'labels':'DetectionPhase:Field'}},
                        {'match':{'labels': 'Priority:Critical'}},
                        {'match':{'labels':'Status:Approved'}}
                    ]
            }
            
        }
}

all_results = es.search(index ='issues_1', body =doc,scroll = '1h')

In [63]:
sid = all_results['_scroll_id']
scroll_size = all_results['hits']['total']

In [64]:
scroll_size

3

In [65]:
final_location = get_final_location(all_results, scroll_size,sid)
final_location
#issue 12 with no longitude hence only 2 records inside locations list

records fetched 2


[[41.917164, -87.686965], [41.891551, -87.607375]]

# Part 2. 3)Heatmap for DetectionPhase is Field AND Priority is Critical AND Status is Approved

In [66]:
heatmap = get_folium_heatmap(final_location)
heatmap

# Part 2. 4)DetectionPhase is Field AND Priority is Critical or High AND Status is Approved or inProgress

In [67]:
doc = {
  "query": {
    "bool": {
      "must": [
        {
          "match": {
            "labels": "DetectionPhase:Field"
          }
        },
        {
          "bool": {
            "should": [
              {
                "match": {
                  "labels": "Priority:Critical"
                }
              },
              {
                "match": {
                  "labels": "Priority:High"
                }
              }
            ],
            "minimum_should_match": "1"
          }
        },
        {
          "bool": {
            "should": [
              {
                "match": {
                  "labels": "Status:Approved"
                }
              },
              {
                "match": {
                  "labels": "Status:inProgress"
                }
              }
            ],
            "minimum_should_match": "1"
          }
        }
      ]
    }
  }
}

all_results = es.search(index ='issues_1', body =doc,scroll = '1h')

In [68]:
sid = all_results['_scroll_id']
scroll_size = all_results['hits']['total']

In [69]:
scroll_size

24

In [70]:
final_location = get_final_location(all_results, scroll_size,sid)
#although 23 records are there but only 4 issues have longitude and latitude
#issue#12 had only latitude and no longitude
final_location

records fetched 4


[[41.89323, -87.617419],
 [41.853136, -87.63316],
 [41.917164, -87.686965],
 [41.891551, -87.607375]]

# Part 2. 4)DetectionPhase is Field AND Priority is Critical or High AND Status is Approved or inProgress

In [71]:

heatmap = get_folium_heatmap(final_location)
heatmap

# Part 2. 5) All locations that got at least 5 issues on the same location

In [72]:
#creating a new index since we would need Latitude and Longitude fields outside of the labels array
final_list = []
for data in df1:
    data2 = data
    temp2 = {}
    for key, values in data2.items():
        if(isinstance(values,list)):
            for i in values:
                if(i.split(":")[0].strip().lower()=='longitude' or i.split(":")[0].strip().lower() == 'latitude'):
                    temp2[i.split(":")[0].strip()]=i.split(":")[1].strip()
                
    data.update(temp2)
    action = {
    "_index": "issues_3",
    "_type": "issues",
    "_id": data["issue_number"],
    "_source":data           #temp2
    }
    final_list.append(action)

helpers.bulk(es,final_list)

(255, [])

In [73]:
doc = {
    "aggs": {
        "top_tags": {
            "terms": {
                "field": "Latitude.keyword",
                #"size": 5
                "min_doc_count" : 5
            },
            "aggs": {
                "top_sales_hits": {
                    "top_hits": {
                        "_source": {
                            "includes": [ "Latitude", "Longitude" ]
                        },
                        "size" : 1
                    }
                }
            }
        }
    }
}

all_results = es.search(index ='issues_3', body =doc,scroll = '1h')
#all_results

In [74]:
final_location_ll = []
latitude,longitude = 0, 0 
for i in all_results['aggregations']['top_tags']['buckets']:
    latitude = float(i['top_sales_hits']['hits']['hits'][0]['_source']['Latitude'])
    longitude = float(i['top_sales_hits']['hits']['hits'][0]['_source']['Longitude'])
    final_location_ll.append([latitude,longitude])
final_location_ll[:10]

[[41.847095, -87.616767],
 [41.877817, -87.631247],
 [41.883772, -87.625962],
 [41.852623, -87.611958],
 [41.89302, -87.631556],
 [41.89323, -87.617419],
 [41.877846, -87.631296],
 [41.880982, -87.630553],
 [41.853136, -87.63316],
 [41.917164, -87.686965]]

# Part 2. 5) All locations that got at least 5 issues on the same location

In [75]:

heatmap = get_folium_heatmap(final_location_ll)
heatmap